In [1]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol without an attacker.

# produces the entangled state 1/sqrt(2) * (|01> - |10>)
def entangled_pair():
    q = QuantumCircuit(2)
    q.h(0)
    q.cx(0,1)
    q.x(1)
    q.z(0)
    return q

def random_3():
    global root_2, root_3
    q = QuantumCircuit(1)
    op = Operator([[1/root_3 , -1*root_2/root_3],[root_2/root_3, 1/root_3]])
    q.unitary(op,0) 
    q.measure_all() 
    backend = BasicSimulator()
    compiled = transpile(q, backend)
    job_sim = backend.run(compiled, shots=1)
    result_sim = job_sim.result() 
    counts = result_sim.get_counts(compiled)
    return counts.get("1",0)

def random_2():
    q = QuantumCircuit(1) 
    q.h(0) 
    q.measure_all() 
    backend = BasicSimulator()
    compiled = transpile(q, backend)
    job_sim = backend.run(compiled, shots=1)
    result_sim = job_sim.result() 
    counts = result_sim.get_counts(compiled)
    return counts.get("1",0)

def get_basis():
    # get one basis with 1/3 chance
    # obtain bit 0 with 1/3 chance
    basis = random_3()
    if basis != 0:
        # if not then obtain bit 1 with 1/2 chance overall chance for each is 1/3
        basis +=  random_2() 
    return basis

# defining operators for use in E91

# need to create W and V operators
root_2 = math.sqrt(2)
root_3 = math.sqrt(3)
denom_1 = math.sqrt(4 + 2*root_2)
denom_2 = math.sqrt(4 - 2*root_2) 

# w = 1/sqrt(2) * (X + Z)
W = [[(-1 + root_2)/denom_2, (-1)/denom_2],
     [(1 + root_2)/denom_1, 1/denom_1]]

W = Operator(W)

# v = 1/sqrt(2) * (X-Z) 
V = [ [  1 / denom_1 , (1 + root_2) / denom_1 ],
    [ -1 / denom_2 , (root_2 - 1) / denom_2 ] ]

V = Operator(V) 







In [3]:


# Create entangles state
q_e91 = entangled_pair()
# Alice applies her basis
q_e91.unitary(W,[0])
# Bob applies his basis
q_e91.unitary(W,[1])
# measure 
q_e91.measure_all()
backend = BasicSimulator()
compiled = transpile(q_e91, backend)
job_sim = backend.run(compiled, shots=10)
result_sim = job_sim.result() 
counts = result_sim.get_counts(compiled)
count00 = counts.get("00",0) 
count01 = counts.get("01",0) 
count10 = counts.get("10",0) 
count11 = counts.get("11",0) 
# The return value includes the conversion from measurement results 0,1 to +1,-1
# Each 00 means a value of  1 (+1 * +1)
# Each 01 means a value of -1 (+1 * -1)
# Each 10 means a value of -1 (-1 * +1)
# Each 11 means a value of  1 (+1 * +1)

print(counts)


{'10': 8, '01': 2}


In [4]:
N = 20

Alice_bits = []
Bob_bits = []
Alice_bases = []
Bob_bases = []
wrong_base_sum = 0
shared_key_bits = []


for i in range(int(9*N/2)):
    Alice_bases.append(get_basis())
    Bob_bases.append(get_basis())


for i in range(int(9*N/2)):
    ab_q = entangled_pair()
    # Alice applies her operator
    if Alice_bases[i] == 1:
        ab_q.unitary(W,1)
    elif Alice_bases[i] == 0:
        ab_q.h(1)
    # Bob applies his operator
    if Bob_bases[i] == 0:
        ab_q.unitary(W,0)
    elif Bob_bases == 2:
        ab_q.unitary(V,0)
    # Bob inverts his bit as previously agreed
    ab_q.x(0)
    ab_q.measure_all()
    backend = BasicSimulator()
    compiled = transpile(ab_q, backend)
    job_sim = backend.run(compiled, shots=1)
    result_sim = job_sim.result() 
    counts = result_sim.get_counts(compiled)
    count00 = counts.get("00",0) 
    count01 = counts.get("01",0) 
    count10 = counts.get("10",0) 
    count11 = counts.get("11",0) 
    if (Alice_bases[i] == 1 and Bob_bases[i] == 0) or (Alice_bases[i] == 2 and Bob_bases[i] == 1):
        if(count00 > 0):
            shared_key_bits.append(0)
        else:
            shared_key_bits.append(1)
    else:
        if(Alice_bases[i] == 0 and Bob_bases[i] == 2):
            wrong_base_sum -= 1
        else:
            wrong_base_sum += 1

print(abs(wrong_base_sum/N))

print(abs(wrong_base_sum/N) - 2**(3/2))

print(len(shared_key_bits))

        
        
        
        


2.8
-0.02842712474619047
18


In [5]:
def E91(N):
    Alice_bases = []
    Bob_bases = []
    wrong_base_sum = 0
    shared_key_bits = []
    entangled = True
    
    
    for i in range(int(9*N/2)):
        Alice_bases.append(get_basis())
        Bob_bases.append(get_basis())


    for i in range(int(9*N/2)):
        ab_q = entangled_pair()
        # Alice applies her operator
        if Alice_bases[i] == 1:
            ab_q.unitary(W,1)
        elif Alice_bases[i] == 0:
            ab_q.h(1)
        # Bob applies his operator
        if Bob_bases[i] == 0:
            ab_q.unitary(W,0)
        elif Bob_bases == 2:
            ab_q.unitary(V,0)
        # Bob inverts his bit as previously agreed
        ab_q.x(0)
        ab_q.measure_all()
        backend = BasicSimulator()
        compiled = transpile(ab_q, backend)
        job_sim = backend.run(compiled, shots=1)
        result_sim = job_sim.result() 
        counts = result_sim.get_counts(compiled)
        count00 = counts.get("00",0) 
        # If correct basis for inverse,identical after Bob inversion, bit values
        if (Alice_bases[i] == 1 and Bob_bases[i] == 0) or (Alice_bases[i] == 2 and Bob_bases[i] == 1):
            if(count00 > 0):
                shared_key_bits.append(0)
            else:
                shared_key_bits.append(1)
            
        # Otherwise add to the running counter of values
        else:
            # as per formula one combination has negative sign
            if(Alice_bases[i] == 0 and Bob_bases[i] == 2):
                wrong_base_sum -= 1
            else:
                wrong_base_sum += 1
    # work out S
    S = abs(wrong_base_sum/N)
    # check if output S is suitably close to 2sqrt(2) for entanglement test
    if S <= 2.0:
        entangled = False
        
    return shared_key_bits, S, entangled

key, S, ent = E91(20)

print(len(key))
print(S)
print(ent)


    

15
2.55
True
